In [1]:
import pandas as pd
import logging
import ast
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import Dict
import re

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
import google.generativeai as genai
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

llm =  genai.GenerativeModel("gemini-2.0-flash")


In [4]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = chroma_client.get_or_create_collection(name="documents")

In [60]:
def retriever(question):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = model.encode([question], convert_to_tensor=True).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3
    )
    return results

def format_docs(docs):
    return "\n\n".join(['\n'.join([res['name'],res["description"]]) for res in docs["metadatas"][0]])
# Example query
docs = retriever("Nốt hương của Bleu Channel?")

print("Generated Response:", format_docs(docs))

Generated Response: Bleu de Chanel
Hương đầu: Quả bưởi, Quả chanh vàng, Bạc hà, Tiêu hồng. Hương giữa: Gừng, Nhục đậu khấu, Hoa nhài, Iso E Super. Hương cuối: Nhang, Cỏ hương bài, Gỗ tuyết tùng, Gỗ đàn hương, Hoắc hương, Nhựa hương Labdanum, Xạ hương trắng. Chanel Bleu de Chanel đã đem tới thế giới hương thơm một mùi hương dễ chịu. Cam chanh mở màn bằng nét đặc trưng của nốt hương Bưởi, tươi sáng mọng nước cùng Gừng thanh lịch sạch sẽ, rồi nhẹ nhàng thoang thoảng chút hương hoa. Gừng của Bleu de Chanel rõ ràng, dễ gần, đẹp và nịnh mũi. Lắng dần xuống, mùi hương của Bleu de Chanel trở nên dày và có chiều sâu hơn với Nhang mềm ngọt, cùng sự rắn chắc vững chãi của hương Gỗ, qua hương thơm từ Cỏ hương bài, Gỗ tuyết tùng và Đàn hương. Bạn thấy chứ? Không cần bàn quá nhiều về mùi hương của Bleu de Chanel, một hương thơm dễ mến và đa dụng. Sự thành công của Bleu de Chanel thậm chí còn lớn đến mức tạo thành một trào lưu những mùi hương đi theo phong cách của chai nước hoa này. Tươi sáng, dễ ch

In [6]:
class RunnableChain:
    """Lớp bọc lambda function để hỗ trợ .invoke()"""
    def __init__(self, func):
        self.func = func

    def invoke(self, **kwargs):
        return self.func(**kwargs)

    def __or__(self, other):
        """Hỗ trợ chaining nhiều bước"""
        if not callable(other):
            raise TypeError(f"Cannot chain with type {type(other)}")

        # Ensure the next step is also a RunnableChain
        return RunnableChain(lambda **kwargs: other.invoke(self.func(**kwargs)))


class ChatPromptTemplate:
    def __init__(self, template: str):
        self.template = template
        self.variables = self.extract_variables(template)

    def extract_variables(self, template: str):
        return re.findall(r"\{(\w+)\}", template)

    def format(self, **kwargs) -> str:
        """Điền giá trị vào template"""
        missing_vars = [var for var in self.variables if var not in kwargs]
        if missing_vars:
            raise ValueError(f"Missing variables: {missing_vars}")

        return self.template.format(**kwargs)

    @classmethod
    def from_template(cls, template: str):
        return cls(template)

    def __or__(self, model):
        if not hasattr(model, "generate_content"):
            raise TypeError(f"Cannot chain with type {type(model)}")

        return RunnableChain(lambda **kwargs: model.generate_content(self.format(**kwargs)))

In [7]:

template = """Using the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

formatted_prompt = prompt.format(context="AI is evolving rapidly", question="What is AI?")
print(formatted_prompt)

Using the following context:
AI is evolving rapidly

Question: What is AI?



In [8]:
class RunnableFunction:
    def __init__(self, func):
        self.func = func  

    def __or__(self, other):
        if not callable(other):
            raise TypeError(f"Cannot chain with type {type(other)}")
        return RunnableFunction(lambda x: other(self.func(x)))

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def invoke(self, x):
        return self.func(x)


def str_output_parser(output):
    if hasattr(output, "text"):
        return output.text  # Nếu output có thuộc tính `.text`
    if isinstance(output, dict) and "text" in output:
        return output["text"]  # Nếu output là dict chứa key "text"
    return str(output)  # Chuyển tất cả về string


In [21]:
template = """Using the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Khởi tạo parser
parser = RunnableFunction(str_output_parser)

# Xây dựng chain
chain = (
    prompt 
    | llm 
    | parser
)

question="Nốt hương của Bleu Chanel là gì?"

docs = retriever(question)

# Chạy pipeline
response = chain.invoke(context=format_docs(docs), question=question)
print(response)


Có vẻ như bạn đang hỏi về nốt hương của phiên bản Bleu de Chanel gốc (Eau de Toilette). Dưới đây là các nốt hương:

*   **Hương đầu:** Quả bưởi, Quả chanh vàng, Bạc hà, Tiêu hồng.
*   **Hương giữa:** Gừng, Nhục đậu khấu, Hoa nhài, Iso E Super.
*   **Hương cuối:** Nhang, Cỏ hương bài, Gỗ tuyết tùng, Gỗ đàn hương, Hoắc hương, Nhựa hương Labdanum, Xạ hương trắng.


# Khám phá sở thích, nhu cầu của khách hàng.


## Tạo cuội hội thoại với khách hàng

In [61]:
prompt_begin_template = """
Bạn là trợ lý ảo của Namperfume, chuyên tư vấn nước hoa. Hãy bắt đầu cuộc trò chuyện một cách tự nhiên và đặt câu hỏi một cách linh hoạt để hiểu rõ sở thích của khách hàng. Dưới đây là một số gợi ý về thông tin bạn có thể thu thập:

* Mùi hương khách hàng yêu thích là gì? (Ví dụ: tươi mát, hoa cỏ, gỗ, trái cây, vani,...)
* Xác định giới tính của khách hàng sử dụng? Mua cho bản thân hay mua tặng cho ai?
* Khách hàng muốn sử dụng nước hoa cho mục đích gì? (Hàng ngày, dịp đặc biệt, đi làm, hẹn hò,...)
* Phong cách mà khách hàng hướng đến là gì? (Năng động, thanh lịch, lãng mạn, cá tính, quyến rũ,...)
* Khách hàng mong muốn nước hoa mang lại cảm giác gì? (Tự tin, thư giãn, nổi bật, ấm áp,...)
* Thời tiết hoặc mùa nào mà khách hàng thường sử dụng nước hoa?
* Ngân sách mà khách hàng dự định chi cho nước hoa là bao nhiêu?
* Khách hàng có thương hiệu nước hoa yêu thích nào không?

**Lưu ý quan trọng:**

* Không nhất thiết phải hỏi tất cả các câu hỏi. Hãy lắng nghe câu trả lời của khách hàng và đặt câu hỏi tiếp theo một cách tự nhiên dựa trên thông tin đã có.
* Nếu bạn cảm thấy đã thu thập đủ thông tin để đưa ra gợi ý phù hợp, hãy chuyển sang bước giới thiệu sản phẩm mà không cần hỏi thêm.
* Ưu tiên tạo ra một cuộc trò chuyện thoải mái và hữu ích cho khách hàng.
* Chỉ hỏi chưa tư vấn sản phẩm cụ thể nào.

######
ChatBot: Chào bạn, mình là trợ lý ảo của Namperfume. Rất vui được hỗ trợ bạn tìm kiếm mùi hương ưng ý. Bạn có thể cho mình biết bạn đang tìm kiếm loại nước hoa như thế nào không?

Khách hàng: {question}

ChatBot:
"""


prompt_begin = ChatPromptTemplate.from_template(prompt_begin_template)
input_prompt_begin = prompt_begin.format(question="Tôi muốn được tư vấn nước hoa")

response = llm.generate_content(input_prompt_begin)
print(f"ChatBot: {str_output_parser(response)}\n")

history = [input_prompt_begin]  # Danh sách lưu lịch sử hội thoại

while True:
    user_input = input("Bạn: ")
    

    if "exit" in user_input.lower():
        print("🔹 Đã thoát khỏi chatbot.")
        break
    
    print(f"Khách hàng: {user_input}\n")

    history.append(f"Khách hàng: {user_input}")
    full_prompt = "\n".join(history)
    
    # Chuỗi hội thoại đầy đủ
    response = llm.generate_content(full_prompt)

    # Xử lý phản hồi
    if hasattr(response, "text"):
        bot_response = response.text
    elif isinstance(response, dict) and "text" in response:
        bot_response = response["text"]
    else:
        bot_response = str(response)

    print(f"{bot_response}\n")
    # Lưu tin nhắn của chatbot vào lịch sử
    history.append(f"{bot_response}")



ChatBot: Tuyệt vời! Để có thể tư vấn cho bạn loại nước hoa phù hợp nhất, bạn có thể chia sẻ một chút về sở thích của mình được không? Ví dụ, bạn thường thích những mùi hương như thế nào? (tươi mát, hoa cỏ, gỗ, trái cây, vani,...)?


Khách hàng: Mình thích mùi hương gỗ, mang lại sự ấp ấm 

Tuyệt vời! Mùi hương gỗ ấm áp là một lựa chọn rất tinh tế. Mình rất vui khi được giúp bạn tìm một chai nước hoa gỗ ưng ý. Để mình hiểu rõ hơn về sở thích của bạn, bạn có thể cho mình biết thêm một chút thông tin được không?

Ví dụ như, bạn muốn mùi hương gỗ này dành cho nam hay nữ? Hoặc bạn muốn sử dụng nó trong dịp nào (hàng ngày, đi làm, hẹn hò...)? Mình sẽ dựa vào đó để đưa ra những gợi ý phù hợp nhất.


Khách hàng: Mình muốn mua nước hoa nam, mình sử dụng hằng ngày

Tuyệt vời! Nước hoa nam hương gỗ dùng hàng ngày là một lựa chọn rất phổ biến và dễ sử dụng. Để mình có thể đưa ra những gợi ý sát với sở thích của bạn hơn, bạn có thể cho mình biết bạn thích phong cách như thế nào không? Ví dụ như năng

## Phân tích sở thích, nhu cầu nước hoa của khách hàng


In [62]:
full_text_history = "\n".join(history)
history_chat = full_text_history.split("######")[1]

print(history_chat)


ChatBot: Chào bạn, mình là trợ lý ảo của Namperfume. Rất vui được hỗ trợ bạn tìm kiếm mùi hương ưng ý. Bạn có thể cho mình biết bạn đang tìm kiếm loại nước hoa như thế nào không?

Khách hàng: Tôi muốn được tư vấn nước hoa

ChatBot:

Khách hàng: Mình thích mùi hương gỗ, mang lại sự ấp ấm 
Tuyệt vời! Mùi hương gỗ ấm áp là một lựa chọn rất tinh tế. Mình rất vui khi được giúp bạn tìm một chai nước hoa gỗ ưng ý. Để mình hiểu rõ hơn về sở thích của bạn, bạn có thể cho mình biết thêm một chút thông tin được không?

Ví dụ như, bạn muốn mùi hương gỗ này dành cho nam hay nữ? Hoặc bạn muốn sử dụng nó trong dịp nào (hàng ngày, đi làm, hẹn hò...)? Mình sẽ dựa vào đó để đưa ra những gợi ý phù hợp nhất.

Khách hàng: Mình muốn mua nước hoa nam, mình sử dụng hằng ngày
Tuyệt vời! Nước hoa nam hương gỗ dùng hàng ngày là một lựa chọn rất phổ biến và dễ sử dụng. Để mình có thể đưa ra những gợi ý sát với sở thích của bạn hơn, bạn có thể cho mình biết bạn thích phong cách như thế nào không? Ví dụ như năng đ

# Query Transformations

Dựa vào những sở thích, nhu cầu của khách hàng, hãy tạo ra 5 câu mô tả sản phẩm nước hoa phù hợp với khach hàng:

## Tạo ra 5 câu mô tả khác nhau về sản phẩm cho là phù hợp với khách hàng

In [71]:
query_explore_customer_preferces_template = """
Dựa vào lịch sử trò chuyện sau đây, bạn hãy đóng vai trò là một chuyên gia tư vấn nước hoa cao cấp. Nhiệm vụ của bạn là tạo ra **năm mô tả chi tiết và đa dạng** về sở thích, nhu cầu và mong muốn của khách hàng, nhằm mục đích tìm kiếm các mẫu nước hoa phù hợp nhất từ cơ sở dữ liệu vector.

**Mục tiêu:** Bằng cách khai thác các khía cạnh khác nhau trong yêu cầu của khách hàng, bạn sẽ giúp hệ thống vượt qua những hạn chế của phương pháp tìm kiếm tương tự dựa trên khoảng cách đơn thuần, từ đó mang đến kết quả chính xác và phù hợp hơn.

**Lịch sử trò chuyện:**
{history_chat}

**Yêu cầu:**

* Mỗi mô tả cần tập trung vào việc phân tích và làm rõ các yếu tố sau:
    * Giới tính của khách hàng: Nam/Nữ
    * Mùi hương yêu thích: (ví dụ: tươi mát, hoa cỏ, gỗ, trái cây, vani,...)
    * Mục đích sử dụng: (ví dụ: hàng ngày, dịp đặc biệt, đi làm, hẹn hò,...)
    * Phong cách cá nhân: (ví dụ: năng động, thanh lịch, lãng mạn, cá tính, quyến rũ,...)
    * Ngân sách dự kiến: (ví dụ: 1.000.000 đồng - 2.000.000 đồng, 1.500.000 đồng - 2.000.000 đồng, ...)
* Sử dụng ngôn ngữ chuyên nghiệp, tinh tế và giàu hình ảnh để truyền tải chính xác cảm xúc và mong muốn của khách hàng.
* Tạo ra các mô tả có góc nhìn khác nhau, nhấn mạnh vào các khía cạnh khác nhau trong sở thích của khách hàng.
* Đảm bảo mỗi mô tả đều đủ chi tiết để hệ thống có thể hiểu rõ và tìm kiếm hiệu quả trong cơ sở dữ liệu vector.
* Mô tả cần đúng định dạng đầu ra.

**Định dạng đầu ra:**

Mô tả 1: [Mô tả chi tiết với góc nhìn 1]

Mô tả 2: [Mô tả chi tiết với góc nhìn 2]

Mô tả 3: [Mô tả chi tiết với góc nhìn 3]

Mô tả 4: [Mô tả chi tiết với góc nhìn 4]

Mô tả 5: [Mô tả chi tiết với góc nhìn 5]
"""

In [72]:
prompt_query_explore_customer_preferces = ChatPromptTemplate.from_template(query_explore_customer_preferces_template)

# Xây dựng chain
chain = (
    prompt_query_explore_customer_preferces 
    | llm 
    | parser
)

response = chain.invoke(history_chat=history_chat)
print(response)

Mô tả 1: Một quý ông tìm kiếm sự tự tin hàng ngày thông qua hương thơm nam tính. Anh ấy muốn một loại nước hoa hương gỗ ấm áp, không quá phô trương nhưng vẫn đủ sức thu hút và tạo ấn tượng. Hương gỗ phải là nền tảng vững chắc, mang lại cảm giác an toàn và đáng tin cậy. Anh ấy thích những nốt hương gia vị nhẹ nhàng để thêm phần kích thích và chiều sâu, nhưng không quá nồng gắt. Ngân sách từ 1 triệu đến 1 triệu rưỡi, cho phép anh ấy khám phá những lựa chọn hương gỗ chất lượng tốt, phù hợp để sử dụng thường xuyên.

Mô tả 2: Người đàn ông hiện đại, coi trọng sự tự tin và lịch lãm trong mọi hoàn cảnh. Anh ấy đang tìm kiếm một mùi hương gỗ hàng ngày, nhưng không phải loại gỗ thô ráp, mà là một phiên bản tinh tế, được trau chuốt tỉ mỉ. Anh ta khao khát một mùi hương vừa đủ mạnh mẽ để khẳng định bản thân, vừa đủ dễ chịu để không gây khó chịu cho người xung quanh. Một chút tươi mát của cam chanh sẽ là điểm nhấn thú vị, giúp cân bằng sự ấm áp của gỗ và tạo nên một tổng thể hài hòa, phù hợp với k

In [73]:


def split_descriptions_re(text):
  
    descriptions = re.split(r'\n\nMô tả \d+:\s*', text)
    # Xử lý đoạn đầu tiên nếu nó không bắt đầu bằng "Mô tả"
    final_descriptions = []
    for idx,item in enumerate(descriptions):
        if item.startswith("Mô tả"):
            final_descriptions.append(item)
        else:
            final_descriptions.append(f"Mô tả {idx+1}: " + item)
    return final_descriptions


descriptions = split_descriptions_re(response)
descriptions

['Mô tả 1: Một quý ông tìm kiếm sự tự tin hàng ngày thông qua hương thơm nam tính. Anh ấy muốn một loại nước hoa hương gỗ ấm áp, không quá phô trương nhưng vẫn đủ sức thu hút và tạo ấn tượng. Hương gỗ phải là nền tảng vững chắc, mang lại cảm giác an toàn và đáng tin cậy. Anh ấy thích những nốt hương gia vị nhẹ nhàng để thêm phần kích thích và chiều sâu, nhưng không quá nồng gắt. Ngân sách từ 1 triệu đến 1 triệu rưỡi, cho phép anh ấy khám phá những lựa chọn hương gỗ chất lượng tốt, phù hợp để sử dụng thường xuyên.',
 'Mô tả 2: Người đàn ông hiện đại, coi trọng sự tự tin và lịch lãm trong mọi hoàn cảnh. Anh ấy đang tìm kiếm một mùi hương gỗ hàng ngày, nhưng không phải loại gỗ thô ráp, mà là một phiên bản tinh tế, được trau chuốt tỉ mỉ. Anh ta khao khát một mùi hương vừa đủ mạnh mẽ để khẳng định bản thân, vừa đủ dễ chịu để không gây khó chịu cho người xung quanh. Một chút tươi mát của cam chanh sẽ là điểm nhấn thú vị, giúp cân bằng sự ấm áp của gỗ và tạo nên một tổng thể hài hòa, phù hợp 

## Từ 5 câu mô tả trên, thực hiện retrival thông tin các loại nước hoa phù hợp

In [75]:
docs = []
for description in  descriptions:
    docs.append(retriever(description))
docs

[{'ids': [['28462300', '28462312', '33773937']],
  'embeddings': None,
  'documents': [[None, None, None]],
  'uris': None,
  'data': None,
  'metadatas': [[{'brand': 'Cacharel',
     'compare_at_price': 1683000,
     'count_rate': 17,
     'count_rate_1': 0,
     'count_rate_2': 0,
     'count_rate_3': 0,
     'count_rate_4': 5,
     'count_rate_5': 95,
     'description': 'Hương đầu: Xạ hương trắng, Hoa mẫu đơn, Hoa Freesia, Hương xanh, Quả đào, Mận Hương giữa: Hoa huệ, Hoa linh lan, Cỏ non, Hoa nhài, Hoa ngọc lan tây, Hoa hồng Hương cuối: Cà phê, Vanilla, Rau mùi, Gỗ đàn hương, Nhang, Đậu tonka, Gỗ tuyết tùng Đừng bao giờ đánh giá một mùi hương qua thiết kế chai, đó là điều tôi rút ra được sau một thời gian dấn thân vào thế giới mùi hương. Mấu chốt ở việc này là để ta không có bất kỳ một định kiến hay rập khuôn gì về mùi hương trước khi được thử chúng, và Cacharel Noa chính là ấn phẩm giúp tôi nhận ra chân ái đó. Dẫu mang một thiết kế không thể nào đơn giản hơn, nhưng Cacharel Noa t

In [77]:
retriever("Nước hoa nam, có mùi hương nhẹ nhàng")

{'ids': [['10950924', '10952220', '30372923']],
 'embeddings': None,
 'documents': [[None, None, None]],
 'uris': None,
 'data': None,
 'metadatas': [[{'brand': 'Valentino',
    'compare_at_price': 4300000,
    'count_rate': 92,
    'count_rate_1': 0,
    'count_rate_2': 0,
    'count_rate_3': 2,
    'count_rate_4': 1,
    'count_rate_5': 97,
    'description': 'Nốt hương đầu : Cam Bergamot Nốt hương giữa : Hoa Hồng, Hoa Iris Nốt hương cuối : Vani, Cỏ hoắc hương, Da thuộc Khi nói đến vẻ đẹp thuần khiết, tôi lại nghĩ ngay đến Valentino Donna, hương thơm đầy nữ tính, tinh tế, kiêu kì. Tươi mát với Cam Bergamot, giống một cơn gió của mùa hè, lôi cuốn tôi vào một cuộc phiêu lưu hấp dẫn. Như được tái sinh, biến hoá bản thân mình trở thành một người phụ nữ độc lập. Mang theo đó là sự đậm đà, hương Hoa Hồng và Iris, sắp từng tầng một hài hoà với nhau tạo nên một nốt hương phấn quyến rũ đến ngây dại. Tôi ngạc nhiên, không phải bởi sự ngọt ngào của nốt hương đầu mà chính là sự ngọt ngào đầy cốt

# Re-ranking

#  Đề xuất loại nước hoa - thu thập phản hồi của khách hàng

## Nếu khách hàng chôt sản phẩm thì thực hiện chốt đơn

## Nếu khách hàng không ưng ý thì lập lại quy trình dựa trên phản hồi của khách hàng để gợi ý sản phẩm khác.

# Gửi maill chốt đơn.